In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
train_data_path = "/content/drive/MyDrive/Colab Dataset/DR_New/data/train"
test_data_path = "/content/drive/MyDrive/Colab Dataset/DR_New/data/test"

In [7]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image dimensions
img_width, img_height = 224, 224

# Data generators for training and testing
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical')

# Load the ResNet50 model with pre-trained weights (excluding the top classification layer)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add a new classification layer on top of the base model
x = Flatten()(base_model.output)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the new model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model (example, adjust epochs as needed)
epochs = 10

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size)

print("Training finished.")

# Evaluate the model (optional)
loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

Found 3229 images belonging to 2 classes.
Found 361 images belonging to 2 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 37s 252ms/step - accuracy: 0.7435 - loss: 1.5762 - val_accuracy: 0.8608 - val_loss: 0.3599
Epoch 2/10
  1/100 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.9688 - loss: 0.2676

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9688 - loss: 0.2676 - val_accuracy: 0.8807 - val_loss: 0.3182
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 16s 159ms/step - accuracy: 0.8525 - loss: 0.3988 - val_accuracy: 0.8778 - val_loss: 0.3839
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.8438 - loss: 0.5799 - val_accuracy: 0.8835 - val_loss: 0.3031
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 16s 159ms/step - accuracy: 0.8921 - loss: 0.2971 - val_accuracy: 0.8608 - val_loss: 0.3870
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.8750 - loss: 0.3378 - val_accuracy: 0.9091 - val_loss: 0.2568
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 16s 163ms/step - accuracy: 0.8806 - loss: 0.3144 - val_accuracy: 0.9290 - val_loss: 0.2204
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.9688 - loss: 0.1314 - val_accuracy: 0.9261 - val_loss: 0.1981
Epoch 9/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 15s 153ms/step - accuracy: 0.9120 - loss: 0.2377 - val_accurac

In [8]:
import numpy as np
from sklearn.metrics import classification_report
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(img_width, img_height),
    batch_size=1,
    class_mode='categorical')
# Make predictions on the test set
test_predictions = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size)
test_predicted_classes = np.argmax(test_predictions, axis=1)

# Get true labels from the test generator
test_true_classes = test_generator.classes[test_generator.index_array[:test_generator.samples // test_generator.batch_size * test_generator.batch_size]]


# Get class names
class_names = list(test_generator.class_indices.keys())

# Generate classification report
report = classification_report(test_true_classes, test_predicted_classes, target_names=class_names)
print(report)

Found 361 images belonging to 2 classes.
361/361 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step
              precision    recall  f1-score   support

          dr       0.00      0.00      0.00       182
      normal       0.49      0.98      0.66       179

    accuracy                           0.49       361
   macro avg       0.25      0.49      0.33       361
weighted avg       0.24      0.49      0.33       361

